In [ ]:
import pathlib
import requests

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import cluster, decomposition, ensemble, linear_model,
from sklearn import metrics, model_selection, pipeline, preprocessing, utils

# MNIST Dataset

The original [MNIST](http://yann.lecun.com/exdb/mnist/) dataset consists of 70000 28x28 black and white images in 10 classes. There are 60000 training images and 10000 test images.

In [ ]:
# might be different if using Colab or Kaggle
PROJECT_ROOT_DIR = pathlib.Path("..")

DATA_DIR = PROJECT_ROOT_DIR / "data" / "mnist"
DATA_DIR.mkdir(parents=True, exist_ok=True)

RESULTS_DIR = PROJECT_ROOT_DIR / "results" / "mnist"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

CV_FOLDS = 5

### Download and extract the data (if using Colab or Kaggle!)

In [ ]:
URL = "https://github.com/KAUST-Academy/practical-tools-for-machine-learning/blob/october-2022/data/mnist/mnist.parquet?raw=true"

with open(DATA_DIR / "mnist.parquet", 'wb') as f:
    response = requests.get(URL)
    f.write(response.content)


### Load the data

We will load the data using the [Pandas](https://pandas.pydata.org/) library. Highly recommend the most recent edition of [*Python for Data Analysis*](https://learning.oreilly.com/library/view/python-for-data/9781491957653/) by Pandas creator Wes Mckinney for anyone interested in learning how to use Pandas.

In [ ]:
data = pd.read_parquet(DATA_DIR / "mnist.parquet")
features = data.drop("label", axis=1)
target = data.loc[:, "label"]

In [ ]:
features.info()

In [ ]:
target.head()

# Creating a Test Dataset

Before we look at the data any further, we need to create a test set, put it aside, and never look at it (until we are ready to test our trainined machine learning model!). Why? We don't want our machine learning model to memorize our dataset (this is called overfitting). Instead we want a model that will generalize well (i.e., make good predictions) for inputs that it didn't see during training. To do this we hold split our dataset into training and testing datasets. The training dataset will be used to train our machine learning model(s) and the testing dataset will be used to make a final evaluation of our machine learning model(s).


In [ ]:
model_selection.train_test_split?

In [ ]:
SEED = 42
SEED_GENERATOR = np.random.RandomState(SEED)


def generate_seed():
    return SEED_GENERATOR.randint(np.iinfo("uint16").max)

In [ ]:
# split the dataset into training and testing data
_seed = generate_seed()
_random_state = np.random.RandomState(_seed)

train_features, test_features, train_target, test_target = model_selection.train_test_split(
    features,
    target,
    test_size=1e-1,
    random_state=_random_state
)

In [ ]:
train_features.info()

In [ ]:
test_features.info()

Again, if you want to you can write out the train and test sets to disk to avoid having to recreate them later.

In [ ]:
_ = (train_features.join(train_target)
                   .to_parquet(DATA_DIR / "train.parquet", index=False))

_ = (test_features.join(test_target)
                   .to_parquet(DATA_DIR / "test.parquet", index=False))

## Feature Engineering

Feature engineering is one of the most important parts of any machine learning project. There are two main tasks in feature engineering.

* Feature selection: selecting the best subset of features for training. 
* Feature extraction: combining existing features to produce new features for training.
* Feature creation: finding additional data sources to use as features.

Feature engineering is often the most labor intensive part of building a machine learning pipeline and often requires extensive expertise/domain knowledge relevant to the problem at hand. Recently packages such as [featuretools](https://www.featuretools.com/) have been developed to (partially) automate the process of feature engineering.

### Feature Selection

In [ ]:
(train_features.std(axis=0)
               .describe())

In [ ]:
minimum_threshold = 0 # maybe something greater than 0?
useful_train_pixels = train_features.std(axis=0) > minimum_threshold
useful_train_features = train_features.loc[:, useful_train_pixels]

In [ ]:
useful_train_features.info()

### Feature extraction using Principal Component Analysis (PCA)

In [ ]:
decomposition.PCA?

In [ ]:
_seed = generate_seed()

# hyper-parameters
_pca_hyperparameters = {
    "n_components": 0.95,
    "whiten": False,
}

feature_extractor = decomposition.PCA(**_pca_hyperparameters)
extracted_train_features = feature_extractor.fit_transform(useful_train_features)

In [ ]:
extracted_train_features.shape

In [ ]:
extracted_train_features[:, :5]

In [ ]:
extracted_train_features.mean(axis=0)

In [ ]:
extracted_train_features.std(axis=0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
_ = ax.scatter(
    extracted_train_features[:, 0],
    extracted_train_features[:, 1],
    c=train_target,
    alpha=0.05
)
_ = ax.set_xlabel("Component 1", fontsize=15)
_ = ax.set_ylabel("Component 2", fontsize=15)
_ = ax.set_title(type(feature_extractor))
_ = ax.grid(True)

### Exercise: To whiten, or not to whiten?

Take a close look at the doc string for the `decomposition.PCA` algorithm. What happens if you set `n_components` to a number between 0 and 1 (i.e., `n_components=0.95`)? Why might you want to do this? What does setting `whiten=True` do to the output of the algorithm? Re-run the PCA algorithm above setting `whiten=True` to confirm your answer. Why might you want to set `whiten=True`? 

In [ ]:
# insert code here!

### Feature extraction using K-Means

In [ ]:
_seed = generate_seed()

_extractor_hyperparameters = {
    "n_clusters": 10,
    "random_state": np.random.RandomState(_seed),
}

feature_extractor = cluster.KMeans(**_extractor_hyperparameters)

In [ ]:
extracted_train_features = feature_extractor.fit_transform(useful_train_features)

In [ ]:
extracted_train_features.shape

# Select, train, and validate some pipelines

### Perceptron

In [ ]:
linear_model.Perceptron?

In [ ]:
_seed = generate_seed()

_classifier_hyperparameters = {
    "n_jobs": -1,
    "random_state": np.random.RandomState(_seed),
}

ml_pipeline = pipeline.make_pipeline(
    # insert code here!
)

In [ ]:
_ = ml_pipeline.fit(train_features, train_target)

After training the pipeline we can assess its performance on the training data...

In [ ]:
# make predictions
_train_predictions = ml_pipeline.predict(train_features)

# report the accuracy on the training data
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

...and then assess its performance on new data using cross validation.

In [ ]:
CV_FOLDS = 5

_train_predictions = model_selection.cross_val_predict(
    ml_pipeline,
    X=train_features,
    y=train_target,
    cv=CV_FOLDS,
    n_jobs=-1,
    verbose=1
)

# report the accuracy on the cv data
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

You can also use SGD to fit perceptrons as well!

In [ ]:
linear_model.SGDClassifier?

In [ ]:
_seed = generate_seed()

_classifier_hyperparameters = {
    "loss": "perceptron",
    "fit_intercept": True,
    "verbose": 0,
    "random_state": np.random.RandomState(_seed),
}

ml_pipeline = pipeline.make_pipeline(
    # insert code here!
    linear_model.SGDClassifier(**_classifier_hyperparameters)
)

In [ ]:
_ = ml_pipeline.fit(train_features, train_target)

In [ ]:
# make predictions
_train_predictions = ml_pipeline.predict(train_features)

# report the accuracy on the training data
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

In [ ]:
_train_predictions = model_selection.cross_val_predict(
    ml_pipeline,
    X=train_features,
    y=train_target,
    cv=CV_FOLDS,
    n_jobs=-1,
    verbose=1
)

# report the accuracy on the cv data
_report = metrics.classification_report(
    train_target,
    _train_predictions,
)
print(_report)

### Exercise: Early Stopping

Read through the documentation for the `linear_model.SGDClassifier` and implement a Perceptron and train the model using early stopping to control overfitting.

In [ ]:
linear_model.SGDClassifier?

In [ ]:
# insert your code here!

### Exercise: Feature engineering and Gradient Boosted Trees

Combine feature selection, feature engineering usin PCA with Gradient Boosted Trees. How does this compare in terms of computation speed, and accuracy to other methods?

In [ ]:
# insert your code here!

# Evaluate your models on the test dataset

After tweaking your models for a while, you eventually have a system that performs sufficiently well. Now is the time to evaluate the final model on the test set. Remember to re-train your model on the full training data prior to evaluating on the test data.

In [ ]:
estimators = [
    ???
]

for estimator in estimators:
    _ = estimator.fit(train_features, train_target)

In [ ]:
for estimator in estimators:

    # make predictions
    _test_predictions = estimator.predict(test_features)

    # generate a classification report
    _report = metrics.classification_report(
        test_target,
        _test_predictions,
    )
    print(estimator)
    print(_report)


If you did a lot of tuning, the performance will usually be slightly worse than what you measured using cross-validation (because your system ends up fine-tuned to perform well on the validation data and will likely not perform as well on new, unknown datasets). It is not the case in this example, but when this happens you must resist the temptation to tweak the hyperparameters to make the numbers look good on the test set; the improvements would be unlikely to generalize to new data.